In [1]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas import json_normalize

/Users/melihtaki/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [27]:
def getData(path):
    url = "https://api-nprd.traxes.io/hackathon"
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"
    headers = {
            "x-api-key": api_key  # Include the API key in the header
        }
    response = requests.get(url+path, headers=headers)
    
    ## Check response
    if response.status_code == 200:
        return response.json()
    return KeyError  

# Equipment Data

In [ ]:
json_data = getData("/equipment/GetAllTransformers")
flat_data = json_normalize(json_data, sep='_')  # Flatten the nested JSON


# Convert the JSON data to a Pandas DataFrame
at_df = pd.DataFrame(flat_data)
print(at_df.shape)
at_df.head(2)


(17, 16)


,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8
1,50282127,40.0,11,ONAF1,2015,2015-08-07T00:00:00,no wall,15.51,149.18,22.7,46.8,62.8,0.72,1.32,1.16,13.8


In [ ]:
equipment_ids = list(at_df.equipmentId)

all_load_data = []
for equipment in equipment_ids:
    json_data = getData(f"/equipment/GetTransformerLoad?equipmentId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:45:00")
    equipment_load_df = pd.DataFrame(json_data)
    all_load_data.append(equipment_load_df)

load_df = pd.concat(all_load_data)
print(load_df.shape)
load_df.head(2)

(1093457, 3)


,equipmentId,dateTime,load
0,50033827,2023-04-30T22:45:00,10417.08
1,50033827,2023-04-30T23:00:00,10130.00
2,50033827,2023-04-30T23:15:00,9758.25
3,50033827,2023-04-30T23:30:00,9431.92
4,50033827,2023-04-30T23:45:00,9296.45


In [14]:
equipment_ids = list(at_df.equipmentId)

all_temp_data = []
for equipment in equipment_ids:
    json_data = getData(f"/equipment/GetTransformerTemperature?equipmentId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:45:00")
    equipment_temp_df = pd.DataFrame(json_data)
    all_temp_data.append(equipment_temp_df)

temp_df = pd.concat(all_temp_data)
print(temp_df.shape)
temp_df.head(2)

50033827
50282127
50277055
5004625
5004597
24917
33291
5026188
5017873
5017843
76075
76632
5025312
5030256
50250035
5027447
50249457
(997220, 3)


,equipmentId,dateTime,hotspotTemperature
0,50033827,2023-04-30T22:00:00,41.02
1,50033827,2023-04-30T22:15:00,41.38


In [35]:
transformer_data = pd.merge(load_df, temp_df, on=('equipmentId','dateTime'), how='left')
transformer_data.shape
transformer_data.to_csv("../data/transformer_data.csv", index=False)

   equipmentId             dateTime      load           timestamp
0     50033827  2023-04-30T22:45:00  10417.08 2023-04-30 22:45:00
1     50033827  2023-04-30T23:00:00  10130.00 2023-04-30 23:00:00
2     50033827  2023-04-30T23:15:00   9758.25 2023-04-30 23:15:00
3     50033827  2023-04-30T23:30:00   9431.92 2023-04-30 23:30:00
4     50033827  2023-04-30T23:45:00   9296.45 2023-04-30 23:45:00
   equipmentId             dateTime  hotspotTemperature
0     50033827  2023-04-30T22:00:00               41.02
1     50033827  2023-04-30T22:15:00               41.38
2     50033827  2023-04-30T22:30:00               41.38
3     50033827  2023-04-30T22:45:00               40.78
4     50033827  2023-04-30T23:00:00               40.50
   equipmentId             dateTime      load           timestamp  \
0     50033827  2023-04-30T22:45:00  10417.08 2023-04-30 22:45:00   
1     50033827  2023-04-30T23:00:00  10130.00 2023-04-30 23:00:00   
2     50033827  2023-04-30T23:15:00   9758.25 2023-04-30 23:1

# Weather Data

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas import json_normalize
from datetime import datetime
from meteostat import Hourly, Point

# Dictionnaire des villes et leurs coordonnées
locations = {
    'Mechelen': {'lat': 51.0259, 'lon': 4.4776},
    'Romsee': {'lat': 50.6333, 'lon': 5.6667}, 
    'Uccle': {'lat': 50.8003, 'lon': 4.3592},
    'Auvelais': {'lat': 50.4333, 'lon': 4.6333},
    'Nieuwpoort': {'lat': 51.1333, 'lon': 2.7500},
    'Molenbeek-saint-jean': {'lat': 50.8557, 'lon': 4.3169},
    'Lens': {'lat': 50.5500, 'lon': 3.9333},
    'Schaerbeek': {'lat': 50.8667, 'lon': 4.3833},
    'Wilsele': {'lat': 50.9167, 'lon': 4.7167},
    'Anderlecht': {'lat': 50.8333, 'lon': 4.3167},
    'Nivelles': {'lat': 50.5983, 'lon': 4.3283},
    'Montignies-sur-sambre': {'lat': 50.4000, 'lon': 4.4667},
    'Farciennes': {'lat': 50.4333, 'lon': 4.5333}
}

# Dates de début et fin
start = datetime(2023, 4, 30, 22)
end = datetime(2025, 2, 28, 22, 45)

# Liste pour stocker les données météo
all_weather_data = []

# Pour chaque ville
for city, coords in locations.items():
    # Créer un point de données météo
    point = Point(coords['lat'], coords['lon'])
    
    # Récupérer les données horaires
    data = Hourly(point, start, end)
    df = data.fetch()
    
    # Convertir en DataFrame et réinitialiser l'index
    df = df.reset_index()
    
    # Convertir la colonne time en datetime après la lecture
    df['time'] = pd.to_datetime(df['time'])
    
    # Renommer les colonnes
    df = df.rename(columns={
        'temp': 'temperature',
        'rhum': 'relativeHumidityPercentage',
        'wspd': 'windSpeed',
        'time': 'dateTime'
    })
    
    # Sélectionner uniquement les colonnes nécessaires
    df = df[['dateTime', 'temperature', 'relativeHumidityPercentage', 'windSpeed']]
    
    # Ajouter à la liste
    all_weather_data.append(df)

# Concaténer tous les DataFrames
weather_df = pd.concat(all_weather_data)

# Afficher les premières lignes
print(weather_df.shape)
weather_df.head()


KeyboardInterrupt: 

In [28]:
location_ids = list(at_df.locationId)

all_temp_data = []
for locId in location_ids:
    json_data = getData(f"/weather/GetOutsideTemperature?locationId={locId}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:45:00")
    locId_temp_df = pd.DataFrame(json_data)
    all_temp_data.append(locId_temp_df)

temp_df = pd.concat(all_temp_data)
print(temp_df.shape)
temp_df.head(2)

ValueError: DataFrame constructor not properly called!

In [ ]:
#Get all the transformer load data

# API endpoint
url = "https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad"
equipment_data = list(df.equipmentId)
# List to store all the data
all_data = []
## Loop over each equipmentId
for equipment in equipment_data:
    equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?equipmentId={equipment}&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00"

    # Credentials
    username = "team12@traxes.io"
    password = "vbNWCny90468"

    # Your API key (replace with your actual API key)
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

    # Adding the API key to the headers
    headers = {
        "x-api-key": api_key  # Include the API key in the header
    }

    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, auth=HTTPBasicAuth(username, password), headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            item['equipmentId'] = equipment
            all_data.append(item)
    else:
        print(f"Error for equipmentId {equipment_data}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_tran = pd.DataFrame(all_data)

# Print the DataFrame
print(df_tran)


https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=50033827&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=50282127&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=50277055&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=5004625&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=5004597&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=24917&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerLoad?equipmentId=33291&fromDate=2023-04-30T22:45:00&toDate=2025-02-28T22:45:00
https:/

In [ ]:
#Get all the transformer temperature data

# API endpoint
url = "https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature"
equipment_data = list(df.equipmentId)
# List to store all the data
all_data = []
## Loop over each equipmentId
for equipment in equipment_data:
    equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?equipmentId={equipment}&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59"

    # Credentials
    username = "team12@traxes.io"
    password = "vbNWCny90468"

    # Your API key (replace with your actual API key)
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

    # Adding the API key to the headers
    headers = {
        "x-api-key": api_key  # Include the API key in the header
    }

    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, auth=HTTPBasicAuth(username, password), headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            item['equipmentId'] = equipment
            all_data.append(item)
    else:
        print(f"Error for equipmentId {equipment_data}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_temp = pd.DataFrame(all_data)

# Print the DataFrame
print(df_temp)

https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=50033827&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=50282127&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=50277055&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=5004625&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=5004597&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=24917&fromDate=2023-04-30T22:59:59&toDate=2024-12-31T22:59:59
https://api-nprd.traxes.io/hackathon/equipment/GetTransformerTemperature?equipmentId=33291&fromDate=2023-

In [ ]:
#Get all the weather data - temperature

# API endpoint
url = "https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature"
equipment_data = list(df.locationId)
# List to store all the data
all_data = []
## Loop over each equipmentId
for equipment in equipment_data:

    #equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?locationId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00"

    # Credentials
    username = "team12@traxes.io"
    password = "vbNWCny90468"

    # Your API key (replace with your actual API key)
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

    # Adding the API key to the headers
    headers = {
        "x-api-key": api_key  # Include the API key in the header
    }

    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, auth=HTTPBasicAuth(username, password), headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            #item['equipmentId'] = equipment
            all_data.append(item)

    else:
        print(f"Error for equipmentId {equipment_data}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_weather_temp = pd.DataFrame(all_data)

# Print the DataFrame
print(df_weather_temp)

https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=26&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=11&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=19&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=18&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon

In [ ]:
#Get all the weather data - wind speed

# API endpoint
url = "https://api-nprd.traxes.io/hackathon//weather/GetWind"
equipment_data = list(df.locationId)
# List to store all the data
all_data = []
## Loop over each equipmentId
for equipment in equipment_data:

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?locationId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00"

    # Credentials
    username = "team12@traxes.io"
    password = "vbNWCny90468"

    # Your API key (replace with your actual API key)
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

    # Adding the API key to the headers
    headers = {
        "x-api-key": api_key  # Include the API key in the header
    }

    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, auth=HTTPBasicAuth(username, password), headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            #item['equipmentId'] = equipment
            all_data.append(item)
    else:
        print(f"Error for equipmentId {equipment_data}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_weather_wind = pd.DataFrame(all_data)

# Print the DataFrame
print(df_weather_wind)

https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=26&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=11&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=19&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=18&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://ap

In [ ]:
#Get all the weather data - humidity
# API endpoint
url = "https://api-nprd.traxes.io/hackathon//weather/GetHumidity"
equipment_data = list(df.locationId)
# List to store all the data
all_data = []
## Loop over each equipmentId
for equipment in equipment_data:
    #equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?locationId={equipment}&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00"

    # Credentials
    username = "team12@traxes.io"
    password = "vbNWCny90468"

    # Your API key (replace with your actual API key)
    api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

    # Adding the API key to the headers
    headers = {
        "x-api-key": api_key  # Include the API key in the header
    }

    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, auth=HTTPBasicAuth(username, password), headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            #item['equipmentId'] = equipment
            all_data.append(item)
    else:
        print(f"Error for equipmentId {equipment_data}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_weather_humi = pd.DataFrame(all_data)

# Print the DataFrame
print(df_weather_humi)

https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=26&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=11&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=19&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=18&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2025-02-28T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=23&fromDate=2023-04-30T22:00:00&toDat

### Merging the data

In [ ]:
df.head()

,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8
1,50282127,40.0,11,ONAF1,2015,2015-08-07T00:00:00,no wall,15.51,149.18,22.7,46.8,62.8,0.72,1.32,1.16,13.8
2,50277055,40.0,19,ONAF1,2013,2014-08-08T00:00:00,2 walls,15.51,149.18,22.7,46.8,62.8,0.72,1.32,1.16,13.8
3,5004625,40.0,5,ONAF1,2001,2008-01-01T00:00:00,* no info,16.02,223.28,27.8,48.9,65.7,0.62,1.98,1.30,12.9
4,5004597,40.0,5,ONAF1,2001,2008-01-01T00:00:00,* no info,16.15,216.05,25.0,48.9,65.7,0.62,1.98,1.30,12.9


In [ ]:
df_tran.head()

,equipmentId,dateTime,load
0,50033827,2023-04-30T22:45:00,10417.08
1,50033827,2023-04-30T23:00:00,10130.00
2,50033827,2023-04-30T23:15:00,9758.25
3,50033827,2023-04-30T23:30:00,9431.92
4,50033827,2023-04-30T23:45:00,9296.45


In [ ]:
df_load = pd.merge(df, df_tran, on ="equipmentId")

In [ ]:
df_load.head()

,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient,dateTime,load
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T22:45:00,10417.08
1,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:00:00,10130.00
2,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:15:00,9758.25
3,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:30:00,9431.92
4,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:45:00,9296.45


In [ ]:
df_temp.head()

,equipmentId,dateTime,hotspotTemperature
0,50033827,2023-04-30T23:00:00,40.50
1,50033827,2023-04-30T23:15:00,40.50
2,50033827,2023-04-30T23:30:00,39.74
3,50033827,2023-04-30T23:45:00,39.06
4,50033827,2023-05-01T00:00:00,38.61


In [ ]:
df_transformer = pd.merge(df_load, df_temp, on =("equipmentId",'dateTime'))

In [ ]:
df.shape

(17, 16)

In [ ]:
df_tran.shape

(1093457, 3)

In [ ]:
df_load.shape

(1093457, 18)

In [ ]:
df_transformer.shape

(997152, 19)

In [ ]:
df_transformer.to_csv("final.csv")

# Melih

In [ ]:
df_final = pd.merge(df_load, df_temp, on=["equipmentId", "dateTime"])

In [ ]:
df_final.head()

,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient,dateTime,load,hotspotTemperature
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:00:00,10130.00,40.50
1,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:15:00,9758.25,40.50
2,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:30:00,9431.92,39.74
3,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:45:00,9296.45,39.06
4,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-05-01T00:00:00,9160.97,38.61


In [ ]:
df_weather = pd.merge(df_weather_humi, df_weather_temp, on=["locationId", "dateTime"])

In [ ]:
df_weather_humi

,locationId,dateTime,relativeHumidityPercentage
0,26,2023-04-30T22:00:00,68.40
1,26,2023-04-30T23:00:00,70.96
2,26,2023-05-01T00:00:00,74.35
3,26,2023-05-01T01:00:00,77.63
4,26,2023-05-01T02:00:00,80.86
...,...,...,...
273372,2,2025-02-28T18:00:00,81.09
273373,2,2025-02-28T19:00:00,81.36
273374,2,2025-02-28T20:00:00,79.72
273375,2,2025-02-28T21:00:00,84.07


In [ ]:
df_weather.tail()

,locationId,dateTime,relativeHumidityPercentage,temperature
562830,2,2025-02-28T21:00:00,84.07,3.89
562831,2,2025-02-28T22:00:00,88.51,3.06
562832,2,2025-02-28T22:00:00,88.51,3.06
562833,2,2025-02-28T22:00:00,88.51,3.06
562834,2,2025-02-28T22:00:00,88.51,3.06


In [ ]:
df_weather = pd.merge(df_weather, df_weather_wind, on=["locationId", "dateTime"])

In [ ]:
df_weather.tail()

,locationId,dateTime,relativeHumidityPercentage,temperature,windSpeed
1527690,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527691,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527692,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527693,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527694,2,2025-02-28T22:00:00,88.51,3.06,2.73


In [ ]:
df_weather

,locationId,dateTime,relativeHumidityPercentage,temperature,windSpeed
0,26,2023-04-30T22:00:00,68.40,12.77,2.24
1,26,2023-04-30T23:00:00,70.96,12.26,2.05
2,26,2023-05-01T00:00:00,74.35,11.73,1.60
3,26,2023-05-01T01:00:00,77.63,11.35,1.39
4,26,2023-05-01T02:00:00,80.86,10.80,1.13
...,...,...,...,...,...
1527690,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527691,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527692,2,2025-02-28T22:00:00,88.51,3.06,2.73
1527693,2,2025-02-28T22:00:00,88.51,3.06,2.73


In [ ]:
df_final = pd.merge(df_final, df_weather, on=["locationId", "dateTime"])

In [ ]:
df_final

,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,...,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient,dateTime,load,hotspotTemperature,relativeHumidityPercentage,temperature,windSpeed
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,...,0.73,1.48,1.13,11.8,2023-04-30T23:00:00,10130.00,40.50,70.96,12.26,2.05
1,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,...,0.73,1.48,1.13,11.8,2023-05-01T00:00:00,9160.97,38.61,74.35,11.73,1.60
2,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,...,0.73,1.48,1.13,11.8,2023-05-01T01:00:00,12049.23,38.77,77.63,11.35,1.39
3,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,...,0.73,1.48,1.13,11.8,2023-05-01T02:00:00,11077.78,38.67,80.86,10.80,1.13
4,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,...,0.73,1.48,1.13,11.8,2023-05-01T03:00:00,11093.11,37.92,82.36,10.62,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4560499,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,...,0.66,1.34,1.10,14.4,2024-12-31T22:00:00,11690.06,45.53,94.58,5.30,8.15
4560500,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,...,0.66,1.34,1.10,14.4,2024-12-31T22:00:00,11690.06,45.53,94.58,5.30,8.15
4560501,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,...,0.66,1.34,1.10,14.4,2024-12-31T22:00:00,11690.06,45.53,94.58,5.30,8.15
4560502,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,...,0.66,1.34,1.10,14.4,2024-12-31T22:00:00,11690.06,45.53,94.58,5.30,8.15


In [ ]:
df_melih = pd.merge(df_load, df_temp, on=["equipmentId", "dateTime"])
df_melih

,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient,dateTime,load,hotspotTemperature
0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:00:00,10130.00,40.50
1,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:15:00,9758.25,40.50
2,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:30:00,9431.92,39.74
3,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:45:00,9296.45,39.06
4,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-05-01T00:00:00,9160.97,38.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997147,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T21:45:00,12022.30,45.61
997148,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T22:00:00,11690.06,45.53
997149,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T22:15:00,11384.76,44.32
997150,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T22:30:00,11123.16,44.32


In [ ]:
df_melih =

# Merging weather

In [ ]:
# prompt: load final.csv as pandas
import pandas as pd

df_final = pd.read_csv("final.csv")
df_final

,Unnamed: 0,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,heatRunTest_ambiantTemperature,heatRunTest_deltaTopOil,heatRunTest_deltaHotspot,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient,dateTime,load,hotspotTemperature
0,0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:00:00,10130.00,40.50
1,1,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:15:00,9758.25,40.50
2,2,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:30:00,9431.92,39.74
3,3,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-04-30T23:45:00,9296.45,39.06
4,4,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,25.4,54.1,67.4,0.73,1.48,1.13,11.8,2023-05-01T00:00:00,9160.97,38.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997147,997147,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T21:45:00,12022.30,45.61
997148,997148,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T22:00:00,11690.06,45.53
997149,997149,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T22:15:00,11384.76,44.32
997150,997150,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,23.0,51.9,67.7,0.66,1.34,1.10,14.4,2024-12-31T22:30:00,11123.16,44.32


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas import json_normalize

#Get all the weather data - temperature

# API endpoint
url = "https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature"

# Your API key (replace with your actual API key)
api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

# Adding the API key to the headers
headers = {
    "x-api-key": api_key  # Include the API key in the header
}

locations = list(df_final.locationId.unique())


# List to store all the data
all_data = []
## Loop over each equipmentId
for lId in locations:

    #equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?locationId={lId}&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00"


    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            #item['equipmentId'] = equipment
            all_data.append(item)

    else:
        print(f"Error for equipmentId {lId}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_weather_temp = pd.DataFrame(all_data)

# Print the DataFrame
df_weather_temp

https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=26&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=11&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=19&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=18&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetOutsideTemperature?locationId=16&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackatho

,locationId,dateTime,temperature
0,26,2023-04-30T22:00:00,12.77
1,26,2023-04-30T23:00:00,12.26
2,26,2023-05-01T00:00:00,11.73
3,26,2023-05-01T01:00:00,11.35
4,26,2023-05-01T02:00:00,10.80
...,...,...,...
161310,17,2024-12-31T18:00:00,3.30
161311,17,2024-12-31T19:00:00,3.37
161312,17,2024-12-31T20:00:00,3.76
161313,17,2024-12-31T21:00:00,4.30


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas import json_normalize

#Get all the weather data - temperature

# API endpoint
url = "https://api-nprd.traxes.io/hackathon//weather/GetWind"

# Your API key (replace with your actual API key)
api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

# Adding the API key to the headers
headers = {
    "x-api-key": api_key  # Include the API key in the header
}

locations = list(df_final.locationId.unique())


# List to store all the data
all_data = []
## Loop over each equipmentId
for lId in locations:

    #equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?locationId={lId}&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00"


    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            #item['equipmentId'] = equipment
            all_data.append(item)

    else:
        print(f"Error for equipmentId {lId}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_weather_wind = pd.DataFrame(all_data)

# Print the DataFrame
df_weather_wind

https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=26&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=11&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=19&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=18&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=16&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetWind?locationId=2&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://ap

,locationId,dateTime,windSpeed
0,26,2023-04-30T22:00:00,2.24
1,26,2023-04-30T23:00:00,2.05
2,26,2023-05-01T00:00:00,1.60
3,26,2023-05-01T01:00:00,1.39
4,26,2023-05-01T02:00:00,1.13
...,...,...,...
161310,17,2024-12-31T18:00:00,6.05
161311,17,2024-12-31T19:00:00,6.77
161312,17,2024-12-31T20:00:00,6.29
161313,17,2024-12-31T21:00:00,6.93


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas import json_normalize

#Get all the weather data - temperature

# API endpoint
url = "https://api-nprd.traxes.io/hackathon//weather/GetHumidity"

# Your API key (replace with your actual API key)
api_key = "a47c28cc-9401-450f-b052-db23dccb26c5"

# Adding the API key to the headers
headers = {
    "x-api-key": api_key  # Include the API key in the header
}

locations = list(df_final.locationId.unique())


# List to store all the data
all_data = []
## Loop over each equipmentId
for lId in locations:

    #equipment_data = list(df['equipmentId'])  # Get the equipmentId column as a list

    # Construct the URL with the equipmentId
    url_with_params = f"{url}?locationId={lId}&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00"


    print(url_with_params)
    # Make a request with Basic Authentication and API Key in headers
    response = requests.get(url_with_params, headers=headers)


    # Check if the response is successful
    if response.status_code == 200:
        # Get the JSON data from the response
        json_data = response.json()

        # Append the data with equipmentId to the list
        for item in json_data:
            #item['equipmentId'] = equipment
            all_data.append(item)

    else:
        print(f"Error for equipmentId {lId}: {response.status_code}")
        print(response.text)  # Debugging info

# Convert all the collected data into a DataFrame
df_weather_humi = pd.DataFrame(all_data)

# Print the DataFrame
df_weather_humi

https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=26&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=11&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=19&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=5&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=18&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=23&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=16&fromDate=2023-04-30T22:00:00&toDate=2024-12-31T22:00:00
https://api-nprd.traxes.io/hackathon//weather/GetHumidity?locationId=2&fromDate=2023-04-30T22:00:00&toDat

,locationId,dateTime,relativeHumidityPercentage
0,26,2023-04-30T22:00:00,68.40
1,26,2023-04-30T23:00:00,70.96
2,26,2023-05-01T00:00:00,74.35
3,26,2023-05-01T01:00:00,77.63
4,26,2023-05-01T02:00:00,80.86
...,...,...,...
161310,17,2024-12-31T18:00:00,93.95
161311,17,2024-12-31T19:00:00,94.01
161312,17,2024-12-31T20:00:00,94.46
161313,17,2024-12-31T21:00:00,94.65


In [ ]:
df_weather = pd.merge(df_weather_temp, df_weather_wind, on=["locationId", "dateTime"])
df_weather

,locationId,dateTime,temperature,windSpeed
0,26,2023-04-30T22:00:00,12.77,2.24
1,26,2023-04-30T23:00:00,12.26,2.05
2,26,2023-05-01T00:00:00,11.73,1.60
3,26,2023-05-01T01:00:00,11.35,1.39
4,26,2023-05-01T02:00:00,10.80,1.13
...,...,...,...,...
161310,17,2024-12-31T18:00:00,3.30,6.05
161311,17,2024-12-31T19:00:00,3.37,6.77
161312,17,2024-12-31T20:00:00,3.76,6.29
161313,17,2024-12-31T21:00:00,4.30,6.93


In [ ]:
df_weather = pd.merge(df_weather, df_weather_humi, on=["locationId", "dateTime"])
df_weather

,locationId,dateTime,temperature,windSpeed,relativeHumidityPercentage
0,26,2023-04-30T22:00:00,12.77,2.24,68.40
1,26,2023-04-30T23:00:00,12.26,2.05,70.96
2,26,2023-05-01T00:00:00,11.73,1.60,74.35
3,26,2023-05-01T01:00:00,11.35,1.39,77.63
4,26,2023-05-01T02:00:00,10.80,1.13,80.86
...,...,...,...,...,...
161310,17,2024-12-31T18:00:00,3.30,6.05,93.95
161311,17,2024-12-31T19:00:00,3.37,6.77,94.01
161312,17,2024-12-31T20:00:00,3.76,6.29,94.46
161313,17,2024-12-31T21:00:00,4.30,6.93,94.65


In [ ]:
df_final = pd.merge(df_final, df_weather, on=["locationId", "dateTime"])
df_final

,Unnamed: 0,equipmentId,nominalLoad,locationId,coolingType,manufactureYear,commissioningDate,enclosure,heatRunTest_noLoadLosses,heatRunTest_copperLosses,...,heatRunTest_x,heatRunTest_y,heatRunTest_h,heatRunTest_gradient,dateTime,load,hotspotTemperature,temperature,windSpeed,relativeHumidityPercentage
0,0,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,...,0.73,1.48,1.13,11.8,2023-04-30T23:00:00,10130.00,40.50,12.26,2.05,70.96
1,4,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,...,0.73,1.48,1.13,11.8,2023-05-01T00:00:00,9160.97,38.61,11.73,1.60,74.35
2,8,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,...,0.73,1.48,1.13,11.8,2023-05-01T01:00:00,12049.23,38.77,11.35,1.39,77.63
3,12,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,...,0.73,1.48,1.13,11.8,2023-05-01T02:00:00,11077.78,38.67,10.80,1.13,80.86
4,16,50033827,40.0,26,ONAF1,2008,2010-03-17T00:00:00,1 wall,16.37,186.51,...,0.73,1.48,1.13,11.8,2023-05-01T03:00:00,11093.11,37.92,10.62,1.00,82.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249283,997132,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,...,0.66,1.34,1.10,14.4,2024-12-31T18:00:00,15281.92,47.47,3.87,7.27,94.12
249284,997136,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,...,0.66,1.34,1.10,14.4,2024-12-31T19:00:00,14497.30,48.05,4.15,7.33,93.61
249285,997140,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,...,0.66,1.34,1.10,14.4,2024-12-31T20:00:00,13493.01,47.65,4.28,7.48,92.85
249286,997144,50249457,25.0,2,ONAF1,2011,2016-04-15T00:00:00,4 walls,9.50,106.30,...,0.66,1.34,1.10,14.4,2024-12-31T21:00:00,12719.02,46.36,4.61,8.14,93.20


In [ ]:
# prompt: save as csv

# ... (Your existing code) ...

# Save the final DataFrame to a CSV file
df_final.to_csv('final_merged_data.csv', index=False)  # Set index=False to avoid saving the index column
